In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load dataset
dataset = pd.read_csv('Pune_lstm_2.5.csv')
data = dataset.values
data = data.astype('float32')

In [3]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

In [4]:
# Split data into training and testing sets
train_size = int(len(data) * 0.67)
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]

In [5]:
# Create function to create LSTM model
def create_model():
    model = Sequential()
    model.add(LSTM(60, input_shape=(1, 1)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [6]:
# Reshape input data to be 3-dimensional
X_train = np.reshape(train, (train.shape[0], 1, train.shape[1]))
X_test = np.reshape(test, (test.shape[0], 1, test.shape[1]))

In [7]:
# Create and fit the LSTM model
model = create_model()
model.fit(X_train, train, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
993/993 - 10s - loss: 0.0193 - 10s/epoch - 10ms/step
Epoch 2/100
993/993 - 5s - loss: 0.0012 - 5s/epoch - 5ms/step
Epoch 3/100
993/993 - 5s - loss: 9.2958e-04 - 5s/epoch - 5ms/step
Epoch 4/100
993/993 - 4s - loss: 8.6055e-04 - 4s/epoch - 4ms/step
Epoch 5/100
993/993 - 4s - loss: 7.2972e-04 - 4s/epoch - 4ms/step
Epoch 6/100
993/993 - 5s - loss: 7.5323e-04 - 5s/epoch - 5ms/step
Epoch 7/100
993/993 - 5s - loss: 6.4348e-04 - 5s/epoch - 6ms/step
Epoch 8/100
993/993 - 6s - loss: 5.0846e-04 - 6s/epoch - 6ms/step
Epoch 9/100
993/993 - 5s - loss: 4.5312e-04 - 5s/epoch - 5ms/step
Epoch 10/100
993/993 - 5s - loss: 4.0262e-04 - 5s/epoch - 5ms/step
Epoch 11/100
993/993 - 6s - loss: 4.6924e-04 - 6s/epoch - 6ms/step
Epoch 12/100
993/993 - 6s - loss: 3.5751e-04 - 6s/epoch - 6ms/step
Epoch 13/100
993/993 - 6s - loss: 3.9124e-04 - 6s/epoch - 6ms/step
Epoch 14/100
993/993 - 5s - loss: 3.9550e-04 - 5s/epoch - 5ms/step
Epoch 15/100
993/993 - 5s - loss: 3.2220e-04 - 5s/epoch - 5ms/step
Epoch 16/

In [8]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

16/16 [==============================] - 0s 6ms/step


In [9]:
train_predict = scaler.inverse_transform(train_predict)
train = scaler.inverse_transform(train)
test_predict = scaler.inverse_transform(test_predict)
test = scaler.inverse_transform(test)


In [10]:
"""train_rmse = np.sqrt(np.mean(np.power((train - train_predict), 2)))
test_rmse = np.sqrt(np.mean(np.power((test - test_predict), 2)))
print('Train RMSE: %.2f' % train_rmse)
print('Test RMSE: %.2f' % test_rmse)"""
from sklearn.metrics import r2_score
import math
from sklearn.metrics import mean_squared_error
train_rmse = np.sqrt(mean_squared_error(train, train_predict))
test_rmse = np.sqrt(mean_squared_error(test, test_predict))
print('R2 Score_train:', r2_score(train, train_predict))
print('R2 Score_test:', r2_score(test, test_predict))

print('Train RMSE: %.2f' % train_rmse)
print('Test RMSE: %.2f' % test_rmse)

R2 Score_train: 0.9993518616605287
R2 Score_test: 0.9982693344362026
Train RMSE: 0.55
Test RMSE: 0.53


In [11]:
df=pd.DataFrame({'Actual':train.tolist(),'Predicted from lstm':train_predict.tolist()})
df.to_excel('lstm_2.5.xlsx')
df

,Actual,Predicted from lstm
0,[95.2699966430664],[93.70797729492188]
1,[81.5],[80.4995346069336]
2,[88.87000274658203],[87.60709381103516]
3,[91.47000122070312],[90.09408569335938]
4,[89.68000030517578],[88.38309478759766]
...,...,...
988,[26.670000076293945],[26.235427856445312]
989,[31.700000762939453],[31.225799560546875]
990,[23.950000762939453],[23.54361915588379]
991,[14.670001029968262],[14.407537460327148]


In [12]:
df=pd.DataFrame({'Actual':test.tolist(),'Predicted from lstm':test_predict.tolist()})
df.to_excel('lstm_2.5_test.xlsx')
df

,Actual,Predicted from lstm
0,[35.59000015258789],[35.093692779541016]
1,[51.939998626708984],[51.37813186645508]
2,[56.290000915527344],[55.70380401611328]
3,[66.7699966430664],[66.08232116699219]
4,[43.58000183105469],[43.05127716064453]
...,...,...
485,[13.510000228881836],[13.27171516418457]
486,[15.460000038146973],[15.181940078735352]
487,[14.649999618530273],[14.38794231414795]
488,[10.579999923706055],[10.409884452819824]
